In [34]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
# import math
# import EoN
# import random
import pickle
from Phage_api import settings_local as local_settings

In [35]:
_nodes = pd.read_csv('/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung5_Rep1.gem.txt.cell_type.B-cell_ISV2/MST/node.csv').to_numpy()
node_map = {}
for i in range(len(_nodes)):
    node_map[_nodes[i][0]] = _nodes[i][-1]
    # node_map[_nodes[i][0]] = 10 # ---------------------------------------------------------

In [36]:
path = '/home/platform/project/crustdb_platform/crustdb_api/workspace/crustdb_database/topology/Lung/Lung5_Rep1.gem.txt.cell_type.B-cell_ISV2/MST/MST.pkl'
with open(path, 'rb') as handle:
    G = pickle.load(handle)

In [37]:
node_size = nx.pagerank(G, alpha=0.85,weight=None)
sorted_dict = sorted(node_size.items(), key=lambda x: x[1],reverse=True)
labels = {}
for i in sorted_dict[:10]:
    labels[i[0]] = i[0]
root = list(labels.values())[0]

In [38]:
# BFS
def my_hierarchy(G, root, parent = None):
    children = list(G.neighbors(root))
    if not isinstance(G, nx.DiGraph) and parent is not None:
        children.remove(parent)
    child_list = []
    if len(children)!=0:
        for child in children:
            x = my_hierarchy(G, child, parent=root)
            if x:
                child_list.append({
                    'name': child,
                    'value': node_map[child],
                    'children': x,
                })
            else:
                child_list.append({
                    'name': child,
                    'value': node_map[child],
                    # 'children': x,
                })
        return child_list
    else:
        return None

_res = my_hierarchy(G, root)
res = {
    'name': root,
    'value': node_map[root],
    'children': _res,
}
res


{'name': 'CD37',
 'value': 0.0074118414648811,
 'children': [{'name': 'P2RX5',
   'value': 0.0028786348336054,
   'children': [{'name': 'TNFRSF13B',
     'value': 0.0028118521455718,
     'children': [{'name': 'CCL15', 'value': 0.0019650471874318},
      {'name': 'IGKC',
       'value': 0.000992677742445,
       'children': [{'name': 'IGHG2',
         'value': 0.0080530444425056,
         'children': [{'name': 'IGHG1',
           'value': 0.0047372181961188,
           'children': [{'name': 'MZB1',
             'value': 0.0023314602211215,
             'children': [{'name': 'JCHAIN', 'value': 0.0012437510431236},
              {'name': 'XBP1', 'value': 0.0009375181290187}]}]},
          {'name': 'IGHA1', 'value': 0.003533562121489}]}]}]}]},
  {'name': 'ATM',
   'value': 0.0033723536555397,
   'children': [{'name': 'LPAR5',
     'value': 0.0020100754307691,
     'children': [{'name': 'OAS2',
       'value': 0.0055959297099016,
       'children': [{'name': 'BCL2', 'value': 0.004614931169

In [39]:
with open('/home/platform/project/crustdb_platform/crustdb_api/08_networkx_graph_mst_parentchild_relation.pkl', 'wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [40]:
res['children'] = res['children'][0]
res

{'name': 'CD37',
 'value': 0.0074118414648811,
 'children': {'name': 'P2RX5',
  'value': 0.0028786348336054,
  'children': [{'name': 'TNFRSF13B',
    'value': 0.0028118521455718,
    'children': [{'name': 'CCL15', 'value': 0.0019650471874318},
     {'name': 'IGKC',
      'value': 0.000992677742445,
      'children': [{'name': 'IGHG2',
        'value': 0.0080530444425056,
        'children': [{'name': 'IGHG1',
          'value': 0.0047372181961188,
          'children': [{'name': 'MZB1',
            'value': 0.0023314602211215,
            'children': [{'name': 'JCHAIN', 'value': 0.0012437510431236},
             {'name': 'XBP1', 'value': 0.0009375181290187}]}]},
         {'name': 'IGHA1', 'value': 0.003533562121489}]}]}]}]}}